In [0]:
import os,sys
from ipyparallel import Client
from collections import defaultdict
import tempfile
from Bio.SeqIO.QualityIO import FastqGeneralIterator
import socket
from multiprocessing import Pool, Manager
import shutil
import tempfile
from collections import OrderedDict
import pandas as pd
import numpy as np
import pickle
import shutil
from os import listdir as ls
from os import path as op

In [0]:
rootdir = "/gpfs_fs/home/eckertlab/Mitra/SWWP_seq"

In [0]:
cd $rootdir

In [0]:
proc_files = !find /gpfs_fs/home/eckertlab/Mitra/SWWP_seq -name "*.fastq.gz" | grep -v trialRun | grep -v lane7
proc_files = sorted(proc_files)
proc_files

In [0]:
barcode_files = !find . -name "barcodes.txt" | grep -v trialRun

In [0]:
bc_dfs = {}
for b in barcode_files:
    bc_dfs[b] = pd.read_csv(b, sep="\t")

In [0]:
!mkdir /gpfs_fs/home/eckertlab/Mitra/SWWP_seq2

In [0]:
for i in range(1,5):
    !mkdir /gpfs_fs/home/eckertlab/Mitra/SWWP_seq2/lane{i}

In [0]:
for p in proc_files:
    src = p
    dst = p.replace("SWWP_seq", "SWWP_seq2")
    shutil.copy(src, dst)

In [0]:
cd /gpfs_fs/home/eckertlab/Mitra/SWWP_seq2

In [0]:
for key, df in bc_dfs.items():
    d = "/gpfs_fs/home/eckertlab/Mitra/SWWP_seq2"
    df['bc'] = df.barcode2.apply(lambda x: x.replace("CTCTTTCCCTACACGACGCTCTTCCGATCT", "").upper())
    o = os.path.join(d, key)
    df['enz'] = "EcoRI"
    df[['sample.ID', 'bc', 'enz']].to_csv(o, sep="\t", header=False, index=False)

In [0]:
proc_files = !find /gpfs_fs/home/eckertlab/Mitra/SWWP_seq2 -name "*.fastq.gz"
proc_files = sorted(proc_files)
proc_files

In [0]:
def build_gbsx_cmd(fastq, bc, enz):
    cmd = "/home/cfriedline/g/src/jdk1.8.0_92/bin/java -jar /home/cfriedline/g/src/GBSX/GBSX_v1.2.jar --Demultiplexer"
    return "{} -f1 {} -i {} -gzip true -rad true -mb 2 -me 1 -ea {}".format(cmd, fastq, bc, enz), "gbsx"

def write_qsub(workdir, cmd, label, run, cmd_label):
    with open(os.path.join(workdir, "run_{}.sh".format(cmd_label)), "w") as o:
        o.write("""#!/bin/bash
#$ -N {4}{0}
#$ -cwd
#$ -V
#$ -S /bin/bash
#$ -e {4}_{3}_{0}.err
#$ -o {4}_{3}_{0}.out
cd {1}
{2}
""".format(label, workdir, cmd, run, cmd_label))

rootdir = "/gpfs_fs/home/eckertlab/Mitra/SWWP_seq2"
        
        
for p in proc_files:
    run = os.path.basename(os.path.dirname(p))
    label = "mitra"
    bc_file = os.path.join(os.path.dirname(p), "barcodes.txt")
    enz_file = os.path.join(rootdir, "ecori.txt")
    workdir = os.path.dirname(p)
    gbsx_cmd, gbsx_label = build_gbsx_cmd(p, bc_file, enz_file)
    write_qsub(workdir, gbsx_cmd, label, run, gbsx_label)

In [0]:
fastq_files = !find /gpfs_fs/home/eckertlab/Mitra/SWWP_seq2 | grep '.fastq.gz$' | grep -v undet | grep -v 001
fastq_files = sorted(fastq_files, key=lambda x: os.path.basename(x))
len(fastq_files)

In [0]:
fastq_files

In [0]:
samp_dict = {}
for f in fastq_files:
    d = os.path.basename(f).split(".")
    sample = d[0].strip().replace("_dup", "").upper()
    if not sample in samp_dict:
        samp_dict[sample] = []
    samp_dict[sample].append(f)

In [0]:
#AJO880 (lane2,4), BAL807 (lane1: 807, 807_dup), STI1 (1,3)
for k, v in samp_dict.items():
    if len(v) > 1:
        print(k, v)

In [0]:
!mkdir dedupe

In [0]:
cd dedupe

In [0]:
i = 0
for k, v in samp_dict.items():
    out = "{}.fastq".format(k)
    cmd = None
    if len(v) > 1:
        if os.path.exists(out):
            os.remove(out)
        cmd = "zcat {} > {}".format(" ".join(v), out)
    else:
        cmd = "zcat {} > {}".format(v[0], out)
    print(i, k)
    !$cmd 
    i += 1

In [0]:
fastq_files = !find /gpfs_fs/home/eckertlab/Mitra/SWWP_seq2/dedupe | grep '.fastq$'

In [0]:
len(fastq_files)

In [0]:
with open(os.path.join("/gpfs_fs/home/eckertlab/Mitra/SWWP_seq2", "fastq_files.txt"), "w") as o:
    for f in fastq_files:
        o.write("%s\n" % f)